In [ ]:
import pandas as pd
from ete3 import NCBITaxa
ncbi = NCBITaxa()

import pandas as pd

# load uhgg metadata and blast results
bacteria_db_metadata = pd.read_csv(
# '/home/carsonjm/resources/uhgg/genome-all_metadata.tsv'
str(snakemake.input.bacteria_db_metadata)
, sep='\t')

# split lineage column
bacteria_db_metadata[['superkingdom', 'phylum', 'class', 'order','family', 'genus', 'species']] = bacteria_db_metadata['Lineage'].str.split(';', expand=True)
bacteria_db_metadata['superkingdom'] = bacteria_db_metadata['superkingdom'].str.partition('d__')[2]
bacteria_db_metadata['phylum'] = bacteria_db_metadata['phylum'].str.partition('p__')[2]
bacteria_db_metadata['class'] = bacteria_db_metadata['class'].str.partition('c__')[2]
bacteria_db_metadata['order'] = bacteria_db_metadata['order'].str.partition('o__')[2]
bacteria_db_metadata['family'] = bacteria_db_metadata['family'].str.partition('f__')[2]
bacteria_db_metadata['genus'] = bacteria_db_metadata['genus'].str.partition('g__')[2]
bacteria_db_metadata['superkingdom'] = bacteria_db_metadata['superkingdom'].str.partition('_')[0]
bacteria_db_metadata['phylum'] = bacteria_db_metadata['phylum'].str.partition('_')[0]
bacteria_db_metadata['class'] = bacteria_db_metadata['class'].str.partition('_')[0]
bacteria_db_metadata['order'] = bacteria_db_metadata['order'].str.partition('_')[0]
bacteria_db_metadata['family'] = bacteria_db_metadata['family'].str.partition('_')[0]
bacteria_db_metadata['genus'] = bacteria_db_metadata['genus'].str.partition('_')[0]


# load uhgg blast results
blast = pd.read_csv(
    # '/home/carsonjm/CarsonJM/phide_piper/06_VIRUS_HOST/01_crispr_spacers/uneven_coverage/uhgg_spacers_blast.tsv'
    str(snakemake.input.blast)
    ,sep='\t', header=None, index_col=False, names=['query', 'accession', 'identity', 'length' ,'mismatch',
'gap_open', 'query_start', 'query_end', 'sub_start', 'sub_end', 'evalue', 'bitscore', 'query_length', 'sub_length'])

# format blast genome so it can be merged with metadata, then merge
blast['Genome'] = blast['accession'].str.rpartition('_', expand=True)[0]
blast_metadata = blast.merge(bacteria_db_metadata, on='Genome')

# filter to only retain high-quality CRISPR spacer matches
blast_metadata_hq = blast_metadata[(blast_metadata['identity'] >= 
snakemake.params.min_identity
# 1
) & (blast_metadata['length'] >= 
snakemake.params.min_length
# 70
)]


# count total spacer hits at specied taxonomic rank
merged=blast_metadata_hq[['query', 'superkingdom', 'phylum', 'class', 'order', 'family', 'genus']]

blast_count = merged.groupby(by=['query'], as_index=False).count()
blast_counts = blast_count.loc[:,['query', 'superkingdom']]
blast_counts.rename(columns = {'superkingdom':'total_hits'}, inplace = True)
blast_counts_taxonomy = merged.merge(blast_counts, on='query', how='left')



# determine if any genomes have > min_agreement agreement at taxonomic level
def determine_consensus(taxonomic_rank, spacer_table):
    rank_hits = spacer_table.groupby(['query', taxonomic_rank], as_index=False
        ).agg(taxonomic_rank_hits=(taxonomic_rank,'count'))
    rank_hits_merged = spacer_table.merge(rank_hits, on=['query', taxonomic_rank])
    rank_hits_merged[taxonomic_rank + '_percent_agreement'] = rank_hits_merged['taxonomic_rank_hits']/rank_hits_merged['total_hits']
    rank_hits_consensus = rank_hits_merged[rank_hits_merged[taxonomic_rank + '_percent_agreement']*100 >
    snakemake.params.min_agreement
    # 70
    ]
    rank_hits_consensus_first = rank_hits_consensus.groupby(['query', taxonomic_rank], as_index=False).first()
    rank_hits_consensus_first.rename(columns={'taxonomic_rank_hits':str(taxonomic_rank)+'_hits'}, inplace=True)

    return rank_hits_consensus_first

# determine genus level consensus
genus_consensus = determine_consensus('genus', blast_counts_taxonomy)
genus_unannotated = blast_counts_taxonomy[~blast_counts_taxonomy['query'].isin(
    genus_consensus['query'])]

# determine family level consensus
family_consensus = determine_consensus('family', genus_unannotated)
family_consensus['genus'] = 'NA'
family_unannotated = genus_unannotated[~genus_unannotated['query'].isin(
    family_consensus['query'])]

# determine order level consensus
order_consensus = determine_consensus('order', family_unannotated)
order_consensus[['genus', 'family']] = 'NA'
order_unannotated = family_unannotated[~family_unannotated['query'].isin(
    order_consensus['query'])]

# determine class level consensus
class_consensus = determine_consensus('class', order_unannotated)
class_consensus[['genus', 'family', 'order']] = 'NA'
class_unannotated = order_unannotated[~order_unannotated['query'].isin(
    class_consensus['query'])]

# determine phylum level consensus
phylum_consensus = determine_consensus('phylum', class_unannotated)
phylum_consensus[['genus', 'family', 'order', 'class']] = 'NA'
phylum_unannotated = class_unannotated[~class_unannotated['query'].isin(
    phylum_consensus['query'])]

# determine superkingdom level consensus
superkingdom_consensus = determine_consensus('superkingdom', phylum_unannotated)
superkingdom_consensus[['genus', 'family', 'order', 'class', 'phylum']] = 'NA'
superkingdom_unannotated = phylum_unannotated[~phylum_unannotated['query'].isin(
    superkingdom_consensus['query'])]

# merge all results
final_consensus = pd.concat([genus_consensus, family_consensus, order_consensus,
    class_consensus, phylum_consensus, superkingdom_consensus])
final_consensus = final_consensus.fillna('NA')
final_consensus['taxonomy'] = (final_consensus['superkingdom'] + ';'
    + final_consensus['phylum'] + ';' + final_consensus['class'] + ';'
    + final_consensus['order'] + ';' + final_consensus['family'] + ';'
    + final_consensus['genus'])

# format results file and save
final_consensus = final_consensus[['query','total_hits',
'superkingdom','superkingdom_hits','superkingdom_percent_agreement','phylum','phylum_hits','phylum_percent_agreement',
'class','class_hits','class_percent_agreement','order','order_hits','order_percent_agreement',
'family','family_hits','family_percent_agreement','genus','genus_hits','genus_percent_agreement', 'taxonomy']]
final_consensus = final_consensus.add_prefix('blast_')
final_consensus.rename(columns = {'blast_query': 'viral_genome'}, inplace=True)
final_consensus.to_csv(str(snakemake.output.report), index=False)
host_results = final_consensus[['viral_genome', 'blast_taxonomy']]
host_results.to_csv(str(snakemake.output.taxonomy), index=False)
